# Telco Project Part I

## Goals

## Imports

In [1]:
import acquire
import prepare

import numpy as np
import pandas as pd

In [2]:
# Get your telco data
telco_df = acquire.get_telco_data()
print(f'{telco_df.dtypes}')


csv file found and loaded
payment_type_id               int64
internet_service_type_id      int64
contract_type_id              int64
customer_id                  object
gender                       object
senior_citizen                int64
partner                      object
dependents                   object
tenure                        int64
phone_service                object
multiple_lines               object
online_security              object
online_backup                object
device_protection            object
tech_support                 object
streaming_tv                 object
streaming_movies             object
paperless_billing            object
monthly_charges             float64
total_charges                object
churn                        object
contract_type                object
internet_service_type        object
payment_type                 object
dtype: object


In [5]:
# This will show the Data Dictionary
columns = telco_df.columns
features = []
definition = ['Payment type ID','Internet service type ID','Contract type ID','Customer ID','Whether the customer is a male or a female','Whether the customer is a senior citizen or not','Whether the customer has a partner or not','Whether the customer has dependents or not','Number of months the customer has stayed with the company','Whether the customer has a phone service or not','Whether the customer has multiple lines or not','Whether the customer has online security or not',
             'Whether the customer has online backup or not','Whether the customer has device protection or not','Whether the customer has tech support or not','Whether the customer has streaming TV or not','Whether the customer has streaming movies or not','Whether the customer has paperless billing or not','The amount charged to the customer monthly','The total amount charged to the customer','Whether the customer churned or not',
             'The contract term of the customer (Month-to-month, One year, Two year)','Customer’s internet service provider (DSL, Fiber optic, No)','The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))']
for i in columns[0:24]:
    features.append(i)
data_dictionary = pd.DataFrame({'features':features,
                               'definition':definition})

# Set the column width to unlimited
pd.set_option('display.max_colwidth', None)

# Left align the columns
left_aligned_df = data_dictionary.style.set_properties(**{'text-align': 'left'})
left_aligned_df = left_aligned_df.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

display(left_aligned_df)

,features,definition
0,payment_type_id,Payment type ID
1,internet_service_type_id,Internet service type ID
2,contract_type_id,Contract type ID
3,customer_id,Customer ID
4,gender,Whether the customer is a male or a female
5,senior_citizen,Whether the customer is a senior citizen or not
6,partner,Whether the customer has a partner or not
7,dependents,Whether the customer has dependents or not
8,tenure,Number of months the customer has stayed with the company
9,phone_service,Whether the customer has a phone service or not


In [ ]:

# Clean the new dataset using the new function called prep_telco
telco_df = prepare.prep_telco(telco_df)



In [ ]:
# Split your data into train, validate and test
train, validate, test = prepare.split_function(telco_df, 'churn')

In [3]:
train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
5911,8319-QBEHW,Male,0,No,Yes,26,No,No phone service,No,Yes,...,0,1,0,1,0,0,0,0,0,0
2479,3547-LQRIK,Female,0,Yes,No,47,Yes,Yes,No internet service,No internet service,...,0,0,0,1,0,0,1,0,1,0
5889,8280-MQRQN,Female,0,No,No,1,Yes,No,No,No,...,0,1,1,0,0,0,0,0,0,1
6087,8626-PTQGE,Male,0,No,No,69,No,No phone service,Yes,Yes,...,1,1,0,0,1,0,0,0,0,0
785,1142-IHLOO,Female,0,No,No,51,Yes,No,No,No,...,1,0,0,1,0,1,0,1,0,0
